First, install the necessary Python dependencies:

In [ ]:
import sys
!{sys.executable} -m pip install wget --quiet
!{sys.executable} -m pip install openai==1.3.3 --quiet
!{sys.executable} -m pip install sentence-transformers --quiet
!{sys.executable} -m pip install pandas --quiet
!{sys.executable} -m pip install sqlalchemy --quiet
!{sys.executable} -m pip install mysql-connector-python --quiet

Among these, `sentence-transformers` is used for text vectorization, while `sqlalchemy` and `mysql-connector-python` are used for connecting to and performing read/write operations with GreptimeDB. The others are utility libraries.

We're using news articles from the AG corpus as our dataset, containing 2000 records in total.

First, import the necessary dependencies:

In [ ]:
import json  
import os  
import pandas as pd  
import wget  
from sentence_transformers import SentenceTransformer  
import sqlalchemy as sa  
from sqlalchemy import create_engine

Download the model:

In [ ]:
model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

Download and parse the dataset:

In [ ]:
cvs_file_path = 'https://raw.githubusercontent.com/openai/openai-cookbook/main/examples/data/AG_news_samples.csv'  
file_path = 'AG_news_samples.csv'  

if not os.path.exists(file_path):  
    wget.download(cvs_file_path, file_path)  
    print('File downloaded successfully.')  
else:  
    print('File already exists in the local file system.')  

df = pd.read_csv('AG_news_samples.csv')  
data = df.to_dict(orient='records')

Assuming you've correctly installed the standalone version of GreptimeDB following the [installation guide](https://docs.greptime.com/getting-started/installation/greptimedb-standalone/) (or you can use [GreptimeCloud](https://greptime.com/product/cloud) hosted service to test this example), connect to the database via MySQL client or open Dashboard to create the table:

In [ ]:
CREATE TABLE IF NOT EXISTS news_articles (  
    title STRING FULLTEXT,  
    description STRING FULLTEXT,  
    genre STRING,  
    embedding VECTOR(768),  
    ts timestamp default current_timestamp(),  
    PRIMARY KEY(title),  
    TIME INDEX(ts)  
);

Where:

* `title`, `description`, and `genre` correspond to the article's title, description, and type information, all `STRING` type with fulltext indexing for title and description.
* `embedding` is set as a 768-dimensional VECTOR type.
 
GreptimeDB's table model requires a timestamp column (called time index); since our test dataset doesn't have article creation times, we set ts default value to `current_timestamp()`.

Next, let's create embeddings for the dataset's descriptions:

In [ ]:
descriptions = [row['description'] for row in data]  
all_embeddings = model.encode(descriptions)

When using SQL to insert vector data types in GreptimeDB, vectors need to be converted to strings. Here's a function to stringify vector arrays and process the dataset:

In [ ]:
def embedding_s(embedding):  
    return f"[{','.join(map(str, embedding))}]"  
    
for row, embedding in zip(data, all_embeddings):  
    row['embedding'] = embedding_s(embedding)

Connect to the database:

In [ ]:
connection_string = "mysql+mysqlconnector://root:@0.0.0.0:4002/public"  
conn = create_engine(connection_string, echo=True).connect()

In [ ]:
Write data:

In [ ]:
statement = sa.text('''  
    INSERT INTO news_articles (  
        title,  
        description,  
        genre,  
        embedding  
    )  
    VALUES (  
        :title,  
        :description,  
        :label,  
        :embedding  
    )  
''')  

for i in range(0, len(data), 100):  
    conn.execute(statement, data[i:i + 100])

We're writing data to GreptimeDB in batches of 100. If everything goes well, we can try a query in MySQL client or dashboard:

```sql
SELECT title, description, genre, vec_to_string(embedding)   
   FROM news_articles LIMIT 1\G;
```   

Let's try vector search to find articles with similar semantics:

In [ ]:
search_query = 'China Sports'  
search_embedding = embedding_s(model.encode(search_query))  

query_statement = sa.text('''  
    SELECT  
        title,  
        description,  
        genre,  
        vec_dot_product(embedding, :embedding) AS score  
    FROM news_articles  
    ORDER BY score DESC  
    LIMIT 10  
''')  

results = pd.DataFrame(conn.execute(query_statement, dict(embedding=search_embedding)))  
print(results)

Here, we first created an embedding for the search keyword, then used the `vec_dot_product` function to calculate the dot product of vectors as a similarity score and sort by it, limiting output to 10 results.

For more vector functions, refer to the documentation: https://docs.greptime.com/nightly/reference/sql/functions/vector/

Let's also try matching results based on full-text index:


In [ ]:
search_query = 'China Sports'  
query_statement = sa.text('''  
    SELECT  
        title,  
        description,  
        genre  
    FROM news_articles  
    WHERE matches(description, :search_query)  
    LIMIT 10  
''')  

results = pd.DataFrame(conn.execute(query_statement, dict(search_query=search_query)))  
print(results)

The results are different - these are text-based matches where the description contains either 'China' or 'Sports'."